<a href="https://colab.research.google.com/github/wcl20/Imperial-416/blob/master/Tutorial/tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 3: Neural Networks & CNNs

## Running on Colab

In [0]:
! wget https://www.doc.ic.ac.uk/~bglocker/teaching/notebooks/supervised-data.zip
! unzip supervised-data.zip

# data directory
data_dir = 'data/mnist/'

## Running on DoC lab machines

In [0]:
# data directory
data_dir = '/vol/lab/course/416/data/mnist/'

**Aims of this tutorial**:
- Introduce you PyTorch, a library made for working with neural networks.
- Implement Multi-Layer Perceptrons (MLP), starting from linear regression.
- Implement a small Convolutional Neural Network (CNN).
Along the way you will implement forward and back-propagation as well as stochastic gradient descent.

It may look long, but it should be easy to complete. Understanding and familiarizing yourselves with the core points of this tutorial and the PyTorch library is of **very high importance in order to be able to follow the next tutorials and the coursework**. Invest some time to study and understand it, and don't hesitate to ask if you don't understand something.

**Prerequisites**:
- Familiar with python and numpy
- Familiar with logistic regression and MNIST


**Notes**:
- Docs for PyTorch's functions you will need:  
https://pytorch.org/docs/stable/tensors.html  
https://pytorch.org/docs/stable/nn.html  
- Some helper functions are included below for loading and plotting data. They will be used out of the box below.

## Preliminary: Loading and refreshing MNIST

We will be using MNIST data again in this tutorial. Because the images are small, the database allows small networks to be quickly trained using CPU. Anything larger afterwards will require GPUs.

Important point to understand is the structure of the loaded data. Especially the **shape** of the loaded numpy arrays, because we need to manipulate it carefully, when processing it with neural networks.

Lets load and inspect the data...

In [0]:
import numpy as np
import torchvision.datasets as dset


def get_mnist(data_dir, train):
    # data_dir: path to local directory where data is, or should be stored.
    # train: if True, return training data. If False, return test data.
    # download: if data not in data_dir, download it.
    data_set = dset.MNIST(root=data_dir, train=train, transform=None, download=True)
    # Students should only deal with numpy arrays, so that it's easy to follow.
    if train:
        data_x = np.asarray(data_set.train_data, dtype='uint8')
        data_y = np.asarray(data_set.train_labels, dtype='int16') # int64 by default
    else:
        data_x = np.asarray(data_set.test_data, dtype='uint8')
        data_y = np.asarray(data_set.test_labels, dtype='int16') # int64 by default
    return data_x, data_y


def make_lbls_onehot(lbls, num_classes):
    # lbls: np.array of shape [N]
    lbls_onehot = np.zeros(shape=(lbls.shape[0], num_classes ) )
    lbls_onehot[ np.arange(lbls_onehot.shape[0]), lbls ] = 1
    return lbls_onehot


# If datasets are not at specified location, they will be downloaded.
train_imgs, train_lbls = get_mnist(data_dir=data_dir, train=True)
test_imgs, test_lbls = get_mnist(data_dir=data_dir, train=False)

print("[train_imgs] Type: ", type(train_imgs), "|| Shape:", train_imgs.shape, "|| Data type: ", train_imgs.dtype )
print("[train_lbls] Type: ", type(train_lbls), "|| Shape:", train_lbls.shape, "|| Data type: ", train_lbls.dtype )
print('Class labels in train = ', np.unique(train_lbls))

print("[test_imgs] Type: ", type(test_imgs), "|| Shape:", test_imgs.shape, " || Data type: ", test_imgs.dtype )
print("[test_lbls] Type: ", type(test_lbls), "|| Shape:", test_lbls.shape, " || Data type: ", test_lbls.dtype )
print('Class labels in test = ', np.unique(test_lbls))

S_tr_samples = train_imgs.shape[0] # S hereafter. Number of training samples in database.
H_height = train_imgs.shape[1] # H hereafter
W_width = train_imgs.shape[2] # W hereafter
C_classes = len(np.unique(train_lbls)) # C hereafter

Above we see that data have been loaded in *numpy arrays*.    
Arrays with images have **shape ( S = number of samples, H = height, W = width )**.  
Arrays with labels have **shape ( S = number of samples)**, holding one integer per image, the digit's class.

MNIST comprises of a **train set (S_tr = 60000) images** and a **test set (S_te = 10000) images**.  
Model development uses the train set. Finally, a model is evaluated on the test set.

Lets plot a few of them in one collage to have a better look...

In [0]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


def plot_image(image, interpol="nearest", cmap="gray", vminmax=[None,None], figsize=None):
    # image: np.array of one of the following shapes:
    #       grayscale image:    (height, width)
    #       grayscale image:    (height, width, 1)
    #       rgb image:          (height, width, 3)
    print("Plotting image of shape: ", image.shape)
    plt.figure(figsize=figsize) # (figsize=(n_imgs_per_row*0.5, n_rows*0.5)) # size (width, height), in inches.
    if len(image.shape) == 2:
        fig = plt.imshow(image, cmap=cmap, interpolation=interpol, vmin=vminmax[0], vmax=vminmax[1]) # imshow: (w,h) or (w,h,3)
        plt.colorbar(fig)
    elif len(image.shape) == 3 and image.shape[2] == 1:
        fig = plt.imshow(image[:,:,0], cmap=cmap, interpolation=interpol, vmin=v_minmax[0], vmax=vminmax[1]) # imshow: (w,h) or (w,h,3)
        plt.colorbar(fig)
    elif len(image.shape) == 3 and image.shape[2] == 3 :
        _ = plt.imshow(image, interpolation=interpol)
    else:
        raise Error("Wrong shape of given image for plotting.")


def plot_grid_of_images(imgs, n_imgs_per_row=10, interpol="nearest", cmap="gray", vminmax=[None,None]):
    # imgs: numpy array of one of the following shapes:
    #       grayscales images:  (number-of-images, height, width)
    #       grayscales images:  (number-of-images, height, width, 1)
    #       color images:       (number-of-images, height, width, 3)
    n_rows = imgs.shape[0]//n_imgs_per_row + 1*int(imgs.shape[0]%n_imgs_per_row > 0)
    print("n_rows=",n_rows)
    # Append empty images if the last row is not complete
    n_empty_imgs = n_rows * n_imgs_per_row - imgs.shape[0]
    imgs_to_plot = np.concatenate( [imgs, np.zeros((n_empty_imgs, imgs.shape[1], imgs.shape[2]))], axis=0)
    
    # draw row by row
    row_images = [] # each element will be (image-height, image-width X n_imgs_per_row)
    for current_row in range(n_rows):
        tmp_row_images = imgs_to_plot[current_row * n_imgs_per_row : (current_row + 1) * n_imgs_per_row]
        row_images.append( np.concatenate(tmp_row_images, axis=1) )
    # draw all row-images in one image
    collage_of_images = np.concatenate(row_images, axis=0) # array.shape: (height X n_imgs_per_row, width X n_imgs_per_row)
    
    plot_image(collage_of_images, interpol=interpol, cmap=cmap, vminmax=[None,None])


plot_grid_of_images(train_imgs[0:100], n_imgs_per_row=10)

Notice that the intensities in the images take **values from 0 to 255**.

## Preliminary: Data pre-processing

A first step in almost all pipelines is to pre-process the data, to make them more appropriate for a model.

Below, we will perform 3 points:  
a) Change the labels from an integer representation to a **one-hot representation** of the **C=10 classes**.
b) Normalize the **intensities** in the images (from 0-255) to have a **zero-mean** and **unary standard deviation**.
c) **Vectorise the 2D images into 1D vectors for the MLP**, which only gets vector-input per sample. The CNN later will use the original 2D images.  

In [0]:
# a) Change representation of labels to one-hot vectors of length C=10.
train_lbls_onehot = np.zeros(shape=(train_lbls.shape[0], C_classes ) )
train_lbls_onehot[ np.arange(train_lbls_onehot.shape[0]), train_lbls ] = 1
test_lbls_onehot = np.zeros(shape=(test_lbls.shape[0], C_classes ) )
test_lbls_onehot[ np.arange(test_lbls_onehot.shape[0]), test_lbls ] = 1
print("BEFORE: [train_lbls]        Type: ", type(train_lbls), "|| Shape:", train_lbls.shape, " || Data type: ", train_lbls.dtype )
print("AFTER : [train_lbls_onehot] Type: ", type(train_lbls_onehot), "|| Shape:", train_lbls_onehot.shape, " || Data type: ", train_lbls_onehot.dtype )

In [0]:
# b) Normalize image intensities of the database to have mean=0 and std=1 across.
# This commonly facilitates learning:
# - A zero-centered signal with small magnitude allows avoiding exploding/vanishing problems easier.
# - Features of arbitrary & heterogeneous ranges are harder to learn how to model.
def normalize_int_whole_database(data):
    # data: shape [num_samples, H, W, C]
    mu = np.mean(data, axis=(0,1,2), keepdims=True) # Mean int of channel C, over samples and pixels.
    std = np.std(data, axis=(0,1,2), keepdims=True) # Returned shape: [1, 1, 1, C]
    norm_data = (data - mu) / std
    return norm_data

train_imgs = normalize_int_whole_database(train_imgs)
test_imgs = normalize_int_whole_database(test_imgs)

# Lets plot one image.
index = 0 # Try any, up to 60000
print("Plotting image of index: [", index, "]")
print("Class label for this image is: ", train_lbls[index])
print("One-hot label representation: [", train_lbls_onehot[index], "]")
plot_image(train_imgs[index])
# Notice the magnitude of intensities. Black is now negative and white is positive float.

In [0]:
# c) Flatten the images, from 2D matrices to 1D vectors. MLPs take feature-vectors as input, not 2D images.
train_imgs_flat = train_imgs.reshape([train_imgs.shape[0], -1]) # Preserve 1st dim (N = num samples), flatten others.
test_imgs_flat = test_imgs.reshape([test_imgs.shape[0], -1])
print("Shape of numpy array holding the training database:")
print("Original : [S, H, W] = [", train_imgs.shape , "]")
print("Flattened: [S, H*W]  = [", train_imgs_flat.shape , "]")

## Task 1: Stochastic Gradient Descent for Training Neural Nets

Below you are given the main training function, gradient_descent. This will be called by all following parts of the tutorial.  
The function takes a model and data and performs an iteration of gradient descent. Every few steps, it will also predict the testing data and report accuracy.

If the training database is large, processing the whole database per iteration can be very slow. An efficient alternative is **stochastic** gradient descent, where a training batch is randomly sampled per iteration.

In the below, change the code to make gradient descent stochastic, by sampling a **random** batch per iteration instead of constantly the same training samples.

(*Task 1 can be completed independently from Task 2. Try both before and after, to observe the different behaviour.*)


In [0]:
def plot_train_progress(loss_l, acc_train_l, acc_test_l, iters_per_point, total_iters=None):

    fig, axes = plt.subplots(1, 2, sharex=False, sharey=False)
    assert len(loss_l) == len(acc_train_l) == len(acc_test_l)
    x_points = range(0, len(loss_l)*iters_per_point, iters_per_point)
    
    axes[0].plot(x_points, loss_l, color="black", label="Training loss", linewidth=5)
    axes[0].set_title("Training loss", fontsize=10, y=1.022)
    axes[0].yaxis.grid(True, zorder=0)
    axes[0].set_xlabel('Iteration', fontsize=10)
    if total_iters is not None:
        axes[0].set_xlim([0,total_iters])
    axes[0].set_ylim([0,None])
    axes[0].legend(loc='upper right')
    
    axes[1].set_title("Accuracy", fontsize=10, y=1.022)
    axes[1].plot(x_points, acc_train_l, color="blue", label="Train", linewidth=5)
    axes[1].plot(x_points, acc_test_l, color="red", label="Test", linewidth=5)
    axes[1].yaxis.grid(True, zorder=0)
    axes[1].set_xlabel('Iteration', fontsize=10)
    if total_iters is not None:
        axes[1].set_xlim([0,total_iters])
    axes[1].set_ylim([0,100])
    axes[1].legend(loc='lower right')
    
    plt.show()
    
    
def get_random_batch(train_imgs, train_lbls, batch_size, rng):
    # train_imgs: Images for training. Numpy array of shape [S, H, W]
    # train_lbls: Labels of the training images. Numpy array [S], one integer for each of S samples.
    # batch_size: integer. Size that the batch should have.
    
    ####### TODO: Sample a random batch of images for training. Fill in the blanks (???) ######### 
    indices = rng.randint(low=0, high=train_imgs.shape[0], size=batch_size, dtype='int32')
    #indices = range(0, batch_size)
    ##############################################################################################
    
    train_imgs_batch = train_imgs[indices]
    train_lbls_batch = train_lbls[indices]
    return [train_imgs_batch, train_lbls_batch]

def gradient_descent(net, loss_func, update_params_func, rng,
                     train_imgs, train_lbls, test_imgs, test_lbls,
                     batch_size, learning_rate, total_iters, iters_per_test=-1 ):
    # net: Instance of a model. See classes: MLP_Numpy, MLP_Torch, MLP_Torch_Autograd, CNN_Torch_Autograd.
    # loss_func: Function that computes the loss. See functions: cross_entropy_numpy/torch.
    # update_params_func: Function performing SGD parameter updates. See: grad_descent_update_numpy/torch/autograd
    # rng: numpy random number generator
    # train_imgs: The training images. Numpy array, shape [S_tr, H, W]
    # test_imgs: Save as above, for testing images. [S_te, H, W]
    # train_lbls: One hot representation of labels corresponding to train_imgs. Numpy array, shape [S_tr, C]
    # test_lbls: As above, but for testing data. [S_te, C]
    # batch_size: Size N of the batch that should be processed per SGD iteration by a model.
    # learning_rate: self explanatory.
    # total_iters: how many iterations in total to perform.
    # iters_per_test: Integer. Every that many iterations the model predicts the test data and accuracy is reported.
    values_to_plot = {'loss':[], 'acc_train': [], 'acc_test': []}
    
    for t in range(total_iters):
        # Sample batch for this SGD iteration
        train_imgs_batch, train_lbls_batch = get_random_batch(train_imgs, train_lbls, batch_size, rng)
        
        # Forward pass
        y_pred = net.forward_pass(train_imgs_batch)

        # Compute loss: Cross Entropy
        y_real = train_lbls_batch
        loss = loss_func(y_pred, y_real)

        # Backwards pass. Compute gradients.
        grads = net.backward_pass(loss, y_real)

        # Update weights with gradient descent
        update_params_func(net.params, grads, learning_rate=learning_rate)
        
        
        # ==== Report training loss and accuracy ======
        # y_pred and loss can be either np.array, or torch.tensor (see later). If tensor, make it np.array.
        y_pred_numpy = y_pred if type(y_pred) is np.ndarray else y_pred.detach().numpy()
        y_pred_lbls = np.argmax(y_pred_numpy, axis=1) # y_pred is soft/probability. Make it a hard one-hot label.
        y_real_lbls = np.argmax(y_real, axis=1)
        
        acc_train = np.mean(y_pred_lbls == y_real_lbls) * 100. # percentage
        
        loss_numpy = loss if type(loss) is type(float) else loss.item()
        print("[iter:", t, "]: Training Loss: {0:.2f}".format(loss), "\t Accuracy: {0:.2f}".format(acc_train))
        
        # =============== Every few iterations, predict the testing-database ================#
        if t==total_iters-1 or t%iters_per_test == 0:
            y_pred_test = net.forward_pass(test_imgs)
            # ==== Report test accuracy ======
            y_pred_test_numpy = y_pred_test if type(y_pred_test) is np.ndarray else y_pred_test.detach().numpy()
            y_pred_lbls_test = np.argmax(y_pred_test_numpy, axis=1)
            y_real_lbls_test = np.argmax(test_lbls, axis=1)
            acc_test = np.mean(y_pred_lbls_test == y_real_lbls_test) * 100.
            print("\t\t\t\t\t\t\t\t Testing Accuracy: {0:.2f}".format(acc_test))
            
            # Keep list of metrics to plot progress.
            values_to_plot['loss'].append(loss_numpy)
            values_to_plot['acc_train'].append(acc_train)
            values_to_plot['acc_test'].append(acc_test)
    # In the end of the process, plot loss accuracy on training and testing data.
    plot_train_progress(values_to_plot['loss'], values_to_plot['acc_train'], values_to_plot['acc_test'], iters_per_test)
    

## Task 2: Classification with Multi-Layer Perceptron (MLP) in Numpy

We will use Numpy to build a *Multi-Layer Perceptron* for classification, i.e. a neural network (NN) with hidden layer(s).  
In previous tutorial we had used *logistic regression*. An MLP is very similar, but also contains at least a hidden layer.
Lets build this in Numpy. Please consult previous tutorial for Logistic regression.

*(If you get stuck implementing the backward pass in Task 2 or Task 3, you can continue straight with Task 4.)*

**Hint:** To debug all the below sections, it is very useful to print the shapes of arrays/tensors (with `print(tensor_name.shape)`). Follow how they change layer after layer, to double-check that behaviour is as expected.

In [0]:
# -*- coding: utf-8 -*-   
import numpy as np
 
class MLP_Numpy():
    def __init__(self, rng):
        # Construct and initialize network parameters
        D_in = H_height*W_width # Dimension of input feature-vectors. Length of a vectorised image.
        D_hid = 1000 # Dimension of Hidden layer.
        D_out = C_classes # Dimension of Output layer. Number of classes 
        
        ###### TODO: Initialize parameters of MLP by sampling them from N(0,0.01). Fill the blanks #####
        # Also see forward_pass(), to see how they are used and infer correct shapes.
        w1 = rng.normal(loc=0.0, scale=0.01, size=(D_in+1, D_hid))
        w2 = rng.normal(loc=0.0, scale=0.01, size=(D_hid+1, D_out))
        self.params = [w1, w2]
        #################################################################################
        
    def forward_pass(self, batch_imgs):
        # This function applies the model to the input. It essentially defines the model's architecture.
        # Input: batch_imgs: Input batch. Numpy.array of [N=number of samples, H=height, W=width] floats.
        # Returns: y_pred: array of [N, D_out] floats. y_pred[i] contains class posterior (probs) for sample i.
        [w1, w2] = self.params
        
        unary_feature_for_bias = np.ones(shape=(batch_imgs.shape[0], 1)) # [N, 1] column vector.
        x = np.concatenate((batch_imgs, unary_feature_for_bias), axis=1) # Extra feature=1 for bias.
        
        ################ TODO: Fill the blanks (???) ##########################################
        # Build network of form: x -> fully connected -> relu -> fully connected -> softmax 
        # Hidden layer
        h1 = x.dot(w1)
        h1_relu = np.maximum(h1, 0)
        # Fully-connected classifier.
        h1_ext = np.concatenate((h1_relu, unary_feature_for_bias), axis=1) # [N, D_hid+1]
        h2 = h1_ext.dot(w2)
        logits = h2
        ######################################################################################
        
        # Softmax activation function.
        exp_logits = np.exp(logits)
        y_pred = exp_logits / np.sum(exp_logits, axis=1, keepdims=True) 
        # sum with Keepdims=True returns [N,1] array. It would be [N] if keepdims=False.
        # Numpy broadcasts [N,1] to [N,D_out] via repetition, to divide elementwise exp_h2 (which is [N,D_out]).
        
        # Store activations. Will be needed for backprop.
        self.__activations = [x, h1, h1_relu, h1_ext, h2, y_pred]
        
        return y_pred
        
        
    def backward_pass(self, loss, y_real): # NOTE: Loss value is not directly needed for computing grads in closed form.
        # Performs back-propagation in closed form.
        # Computes the gradient of the loss with respect to trainable parameters.
        # y_real: Array [N, D_out]. y_real[i] is one-hot representation of real training label for i-th sample.
        # returns grads: list of arrays, one for each trainable parameter. 
        
        [x, h1, h1_relu, h1_ext, h2, y_pred] = self.__activations
        [w1, w2] = self.params
        
        N = y_pred.shape[0]
        D_out = y_pred.shape[1]
        
        # Derivative of cross entropy wrt input to softmax (straight, without intermediate computes)
        # Difficult derivation. Nice explanation at: https://deepnotes.io/softmax-crossentropy
        grad_logits = (y_pred - y_real) / N 
        
        ##### TODO: Fill in the blanks (???) to complete the back-prop for the 2-layers MLP ###########
        grad_w2 = h1_ext.transpose().dot(grad_logits) # shape: [D_hid+1, D_out]
        grad_h1_ext = grad_logits.dot(w2.transpose()) # shape: [N, D_hid+1]
        grad_h1_relu = grad_h1_ext[:, :-1] # Drop the extra feature dimension for the bias [N, D_hid]
        grad_h1 = grad_h1_relu.copy()
        grad_h1[h1 < 0] = 0
        grad_w1 = x.transpose().dot(grad_h1) # Shape [D_in+1, D_hid]
        ###############################################################################################
        
        # Elements of below list must be in same order as in self.params.
        # Each of the subarrays has the same shape as that of corresponding trainable parameter in self.params.
        grads = [grad_w1, grad_w2]
        return grads
        
        
def cross_entropy_numpy(y_pred, y_real, eps=1e-7):
    # Cross entropy
    # y_pred: Predicted class-posterior probabilities, returned by forward_pass. Matrix of shape [N, D_out]
    # y_real: One-hot representation of real training labels. Same shape as y_pred.
    
    ################# TODO: Complete the calculation of cross-entropy for each sample ################
    x_entr_per_sample = - np.sum( y_real*np.log(y_pred+eps), axis=1) # Sum over classes, axis=1
    ##################################################################################################
    
    loss = np.mean(x_entr_per_sample, axis=0) # Expectation of loss: Mean over samples (axis=0).
    return loss


def grad_descent_update_numpy(params, grads, learning_rate):
    # params: list of trainable parameters, created in initializer of the model.
    # grads: list of gradients, returned by backward pass of model. grads[i] corresponds to params[i]
    # learning_rate: a float.
    assert len(params)==len(grads)
    for i in range(len(params)):
        ############## TODO: Perform the updates of the parameters ###############################
        params[i] = params[i] - learning_rate * grads[i]
        ##########################################################################################
    
# Create the network
SEED = 42
rng = np.random.RandomState(seed=SEED) # Random number generator
net = MLP_Numpy(rng=rng)
# Start training
gradient_descent(net,
                 cross_entropy_numpy, # We are using the Numpy version we wrote above.
                 grad_descent_update_numpy, # We are using the Numpy version we wrote above.
                 rng,
                 train_imgs_flat,
                 train_lbls_onehot,
                 test_imgs_flat,
                 test_lbls_onehot,
                 batch_size=40, 
                 learning_rate=1e-2,
                 total_iters=400,
                 iters_per_test=10)

If you build the MLP correctly, the above should result in a report that shows the model is learning, with training *loss* decreasing towards zero, while training and testing *accuracy* increasing.

When process finishes (400 iterations), metrics will be plotted in the bottom of the report (**scroll down if needed**).

If you have *not* changed gradient_descent() in Task 1, you will see the model overfitting, with training accuracy reaching 100\%, while testing accuracy finishes at 59.57\%.

**Q: Why the overfit, if Task 1 has not been addressed?**  

If you have made gradient descent *stochastic* (random), overfitting will not occur with this small network. But you will notice instability in the plotted metrics. Training and testing accuracy should finish at 87.50\% and 88.15\% respectively.

**Q: What factors cause the instability in the observed metrics, if Task1 has been addressed?**  

# Task 3: MLP in PyTorch

We now introduce a library built for development of deep neural networks, PyTorch. It will be used for the rest of the tutorials and the coursework.  
So, it is very important to understand how it works.

In the below we will write *exactly the same* MLP as in Task 2, but translate it to PyTorch. The changes you should do correspond to those in Task 2, just different (very similar) language. This will show you how similar it is to write Numpy and Pytorch.

In [0]:
# -*- coding: utf-8 -*-
import torch


class MLP_Torch():
    def __init__(self, rng):
        # Construct and initialize network parameters
        D_in = H_height*W_width # Dimension of input feature-vectors. Length of a vectorised image.
        D_hid = 1000 # Dimension of Hidden layer.
        D_out = C_classes # Dimension of Output layer. Number of classes 
        
        ######## TODO: Add a hidden layer, to change this perceptron to an MLP ##########
        # For PyTorch, we first make a numpy array with the *initial* weights...
        w1_init = rng.normal(loc=0.0, scale=0.01, size=(D_in+1, D_hid))
        w2_init = rng.normal(loc=0.0, scale=0.01, size=(D_hid+1, D_out))
        # ... and then we initialize with them the actual *tensor*. This one will change with training.
        w1 = torch.tensor(w1_init, dtype=torch.float)
        w2 = torch.tensor(w2_init, dtype=torch.float)
        self.params = [w1, w2]
        ###########################################################################
        
    def forward_pass(self, batch_imgs):
        # compute predicted y
        [w1, w2] = self.params
        
        batch_imgs_t = torch.tensor(batch_imgs, dtype=torch.float)
        
        unary_feature_for_bias = torch.ones(size=(batch_imgs.shape[0], 1)) # [N, 1] column vector.
        x = torch.cat((batch_imgs_t, unary_feature_for_bias), dim=1) # Extra feature=1 for bias.
        
        #### TODO: Add a hidden layer with ReLU before the classifier #####
        # Hidden layer
        h1 = x.mm(w1)
        h1_relu = h1.clamp(min=0)
        # Fully-connected classifier (aka Densely connected classifier)
        h1_ext = torch.cat((h1_relu, unary_feature_for_bias), dim=1)
        h2 = h1_ext.mm(w2)
        logits = h2
        ######################################################################################
        
        # Softmax activation function.
        exp_logits = torch.exp(logits)
        y_pred = exp_logits / torch.sum(exp_logits, dim=1, keepdim=True) 
        # sum with Keepdim=True returns [N,1] array. It would be [N] if keepdim=False.
        # Torch broadcasts [N,1] to [N,D_out] via repetition, to divide elementwise exp_h2 (which is [N,D_out]).

        # Store activations. Will be needed for backprop.
        self.__activations = [x, h1, h1_relu, h1_ext, h2, y_pred]
        
        return y_pred        
        
        
    def backward_pass(self, loss, y_real): # NOTE: It does not require the loss. Grads computed closed form.
        # Performs back-propagation in closed form.
        # Computes the gradient of the loss with respect to trainable parameters.
        # y_real: Array [N, D_out]. y_real[i] is one-hot representation of real training label for i-th sample.
        # returns grads: list of arrays, one for each trainable parameter. 
        
        [x, h1, h1_relu, h1_ext, h2, y_pred] = self.__activations
        [w1, w2] = self.params
        
        y_real = torch.tensor(y_real, dtype=torch.float) if type(y_real) is np.ndarray else y_real
        
        N = y_pred.shape[0]
        D_out = y_pred.shape[1]
        
        # Backprop from loss all the way back, to compute gradients of trainable parameters with respect to loss.
        
        # Derivative of cross entropy wrt input to softmax (straight, without intermediate computes)
        # Difficult derivation. Nice explanation at: https://deepnotes.io/softmax-crossentropy
        grad_logits = (y_pred - y_real) / N 
        
        ##### TODO: Fill in the blanks (???) to complete the back-prop for the 2-layers MLP ###########
        grad_w2 = h1_ext.t().mm(grad_logits) # [D_hid+1, D_out]
        grad_h1_ext = grad_logits.mm(w2.t()) # [N, D_hid+1]
        grad_h1_relu = grad_h1_ext[:, :-1] # Drop the extra feature dimension for the bias [N, D_hid]
        grad_h1 = grad_h1_relu.clone()
        grad_h1[h1 < 0] = 0
        grad_w1 = x.t().mm(grad_h1)
        ################################################################################################
        
        # Elements of below list must be in same order as in self.params.
        # Each of the subarrays has the same shape as that of corresponding trainable parameter in self.params.
        grads = [grad_w1, grad_w2]
        return grads
        

# Same used by Autograd and CNN
def cross_entropy_torch(y_pred, y_real, eps=1e-7):
    # Cross entropy
    # y_pred: Predicted class-posterior probabilities, returned by forward_pass. Numpy array of shape [N, D_out]
    # y_real: One-hot representation of real training labels. Same shape as y_pred.
    
    # If number array is given, change it to a Torch tensor.
    y_pred = torch.tensor(y_pred, dtype=torch.float) if type(y_pred) is np.ndarray else y_pred
    y_real = torch.tensor(y_real, dtype=torch.float) if type(y_real) is np.ndarray else y_real
    
    ######## TODO: (similar to numpy) Complete the calculation of cross-entropy for each sample ###########
    x_entr_per_sample = - torch.sum( y_real*torch.log(y_pred+eps), dim=1) # Sum over classes, axis=1
    #######################################################################################################
    
    loss = torch.mean(x_entr_per_sample, dim=0) # Expectation of loss: Mean over samples (axis=0).
    return loss


def grad_descent_update_torch(params, grads, learning_rate):
    # params: list of trainable parameters, created in initializer of the model.
    # grads: list of gradients, returned by backward pass of model. grads[i] corresponds to params[i]
    # learning_rate: a float.
    assert len(params)==len(grads)
    for i in range(len(params)):
        ############## TODO: Perform the updates of the parameters ###############################
        # Similar to Numpy, but for torch tensors you should use the param.data and grad.data for assignments.
        params[i].data = params[i].data - learning_rate * grads[i].data
        ##########################################################################################
        
# Create the network
rng = np.random.RandomState(seed=SEED) # Random number generator
net = MLP_Torch(rng=rng)
# Start training
gradient_descent(net,
                 cross_entropy_torch, # We are using the Torch version we wrote above.
                 grad_descent_update_torch, # We are using the Torch version we wrote above.
                 rng,
                 train_imgs_flat,
                 train_lbls_onehot,
                 test_imgs_flat,
                 test_lbls_onehot,
                 batch_size=40, 
                 learning_rate=1e-2,
                 total_iters=400,
                 iters_per_test=10)

If the correct changes are done, you should get *exactly* the same results as in Task 2.

**For more material on transitioning from Numpy to PyTorch, you can check this tutorial: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html**

## Task 4: Auto-differentiation (aka Autograd in PyTorch)

Hopefully Task 3 convinced you it's similar writing in Numpy and PyTorch. But then, why bother with PyTorch?

Because it offers auto-differentiation (called autograd in PyTorch). In Task 2 we wrote manually the back-propagation. For this, we need to derive the derivative of each of a network's component, and implement it. It can get extremely tedious for larger nets. Libraries build for Deep Learning (like PyTorch, Tensorflow, Theano, Chainer, etc), offer auto-differentiation: When the forward pass is implemented, the framework keeps track of the operations performed on the tensors. The backend has already implementations of each operation's derivative. To do the backprop, the library's back end will take care of applying the chain-rule automatically.

Below, we implement a new MLP class, inheritting from Task 3 class, and make only the minor changes required to use Autograd in the backwards pass...

In [0]:
# -*- coding: utf-8 -*-
import torch

class MLP_Torch_Autograd(MLP_Torch):
    def __init__(self, rng):
        # Construct and initialize network parameters
        D_in = H_height*W_width # Dimension of input feature-vectors. Length of a vectorised image.
        D_hid = 1000 # Dimension of Hidden layer.
        D_out = C_classes # Dimension of Output layer. Number of classes 
        
        ######## TODO: Add a hidden layer, to change this perceptron to an MLP ##########
        # EXACTLY as above for MLP_Torch (copy-paste)...
        w1_init = rng.normal(loc=0.0, scale=0.01, size=(D_in+1, D_hid))
        w2_init = rng.normal(loc=0.0, scale=0.01, size=(D_hid+1, D_out))
        # ... EXCEPT for passing one more argument!
        # requires_grad=True tells Torch that it will later need to automatically compute grads for this!
        w1 = torch.tensor(w1_init, dtype=torch.float, requires_grad=True)
        w2 = torch.tensor(w2_init, dtype=torch.float, requires_grad=True)
        self.params = [w1, w2]
        ###########################################################################
        
        
    def forward_pass(self, batch_imgs):
        return MLP_Torch.forward_pass(self, batch_imgs) # Calls parent's (MLP_Torch). No change.
        
    def backward_pass(self, loss, y_real):
        ######### TODO: Return automatically computed gradients in a list (ala task1/2) ################
        # We no longer need to write down the backward pass, for parameters were requires_grads=True, 
        # Calling loss.backward(), torch's Autograd automatically computes grads of loss wrt each parameter p,...
        # ... and **puts them in p.grad**. Return them in a list.
        loss.backward()
        grads = [param.grad for param in self.params]
        return grads
        ################################################################################################
        

def grad_descent_update_autograd(params, grads, learning_rate):
    # params: Type: Tensor. List of trainable parameters, created in initializer of the model. 
    # grads:  Type: Tensor. List of gradients, returned by backward pass. grads[i] corresponds to params[i].
    # learning_rate: a float.
    assert len(params)==len(grads)
    for i in range(len(params)):
        
        ############## TODO: Perform the updates of the parameters ###############################
        # Same as function grad_descent_update_torch() previously (copy paste).
        params[i].data -= learning_rate * grads[i].data
        # params[i].data = ????????????????????????????? * grads[i].data
        ##########################################################################################
        
        # IMPORTANT: Weirdness of Torch's Autograd: Need to manually set gradients to zero at end of iteration.
        # ... otherwise loss.backward() keeps accumulating them, and you would get wrong results.
        grads[i].zero_() # Nothing to do here. *** BUT REMEMBER IT FOR FUTURE & COURSEWORK! ***

# Create the network
rng = np.random.RandomState(seed=SEED) # Random number generator
net = MLP_Torch_Autograd(rng=rng)
# Start training
gradient_descent(net,
                 cross_entropy_torch, # Same as before.
                 grad_descent_update_autograd, # Use the above version, with auto-differentiation.
                 rng,
                 train_imgs_flat,
                 train_lbls_onehot,
                 test_imgs_flat,
                 test_lbls_onehot,
                 batch_size=40, 
                 learning_rate=1e-2,
                 total_iters=400,
                 iters_per_test=10)

If completed correctly, the results should be *exactly* the same as in Task 2 and 3.  
**Please note** the implementation detail of zeroing the gradients in the end of function grad_descent_update_autograd(), needed when Autograd is used.


## Task 5: Building a basic Convolutional Neural Network in PyTorch

The predominant neural networks used in computer vision are Convolutional Neural Networks (CNNs). Their most popular components are convolutions, for processing information and deriving more complex features, and pooling layers for aggregating & downsampling the information.

We will below familiarize ourselves with convolutions and pooling in PyTorch. We will change the 2-layers MLP of the previous tasks to a CNN of the form:  
` x -> conv -> bias -> pool -> flatten -> bias -> fully connected layer -> y_pred`  

Aims:
- Understand the corresponding functions of PyTorch.
- Understand how each convolutional/pooling layer changes the dimensions of the input feature maps.  

These points are important for building more complex models in later tutorials/courseworks, so spend some time experimenting.

In [0]:
# -*- coding: utf-8 -*-
import torch
from torch.nn import functional as F

"""
We provide the main API of the conv2d and max_pool2d operations here for convenience:

conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1)
    input: input tensor of shape: (minibatch, in_channels, Height, Width)
    weight: filters of shape: (out_channels, in_channels, kernel's Height, kernel's Width)
    bias: optional bias tensor of shape: (out_channels).
    stride: stride of the convolving kernel. single scalar or (sH, sW).
    padding: zero paddings on both sides of input. Single number or tuple (padH, padW).
    dilation: dilation rate for dilated convolutions (though we don't discuss it here).
    
For docs, search for *conv2d* at: https://pytorch.org/docs/stable/_modules/torch/nn/functional.html
... or write create a new cell in this notebook, and after the above import, write and run: F.conv2d?

F.max_pool2d(input, kernel_size, stride=None, padding=0, ceil_mode=False)
    input: input tensor of shape: (minibatch, in_channels, Height, Width)
    kernel_size: size of the pooling kernel. (Kernel's Height, kernel's Width)
    stride: stride of the pooling kernel as it convolves the input. single scalar or (stride-H, stride-W).
    padding: zero paddings on both sides of input. Single number or tuple (padH, padW).
    ceil_mode: If True, will use `ceil` instead of `floor` to compute output shape when kernel partly runs out of input borders.

For docs, search for *max_pool2d* at: https://pytorch.org/docs/stable/_modules/torch/nn/functional.html
... or write create a new cell in this notebook, and after the above import, write and run: F.max_pool2d?
"""

class CNN_Torch_Autograd():
    def __init__(self, rng):
        # Construct and initialize network parameters, to build a CNN of the form:
        # x -> conv[5x5] -> bias -> maxpool k[2x2],s[2,2] -> flatten -> bias -> fully conn. layer -> y_pred
        D_in = 1 # Channels of input image.
        D_hid = 10
        ######### QUESTION: How is the below number calculated for this architecture? ###################
        # D_in_fc is the length of hidden feature vector per sample, given as input to last fully conn. layer.
        # Why this number? It's architecture dependent.
        # To answer, check the rest of this function and forward_pass() to understand the architecture.
        # Consider how a conv and a pool change the shape of the input. Print shapes in forward_pass to investigate.
        D_in_fc = 1440
        #################################################################################################
        D_out = C_classes # Dimension of Output layer. Number of classes 
        
        ######## TODO: Initialize convolutional kernels and biases ############
        # Initialize conv kernels and biases for CNN of form:
        # ..... x -> conv -> bias -> pool -> flatten -> bias -> fully connected layer -> y_pred
        # Consult the forward pass, to get an idea about the architecture, to help filling this section.
        # Notice: biases are implemented differently in CNNs than MLPs....
        # ... They are implemented by adding a different *trainable* bias to each channel (feature dimension).
        w1_init = rng.normal(loc=0.0, scale=0.01, size=(D_hid, D_in, 5, 5)) # See Conv's API for expected shape.
        b1_init = rng.normal(loc=0.0, scale=0.01, size=(D_hid)) # One bias per channel of hidden layer
        w2_init = rng.normal(loc=0.0, scale=0.01, size=(D_in_fc, D_out)) # Classification layer, similar to MLP.
        b2_init = rng.normal(loc=0.0, scale=0.01, size=(D_out)) # One bias per channel of hidden layer
        w1 = torch.tensor(w1_init, dtype=torch.float, requires_grad=True)
        b1 = torch.tensor(b1_init, dtype=torch.float, requires_grad=True)
        w2 = torch.tensor(w2_init, dtype=torch.float, requires_grad=True)
        b2 = torch.tensor(b2_init, dtype=torch.float, requires_grad=True)
        self.params = [w1, b1, w2, b2]
        #############################################################################################
        
        
    def forward_pass(self, batch_imgs):
        # compute predicted y
        [w1, b1, w2, b2] = self.params
        
        x = torch.tensor(batch_imgs, dtype=torch.float)
        
        #### TODO: Add a Convolutional hidden layer with ReLU before classifier ####################
        # Make CNN: x -> conv[5x5] -> bias -> maxpool k[2x2],s[2,2] -> flatten -> bias -> fully conn. layer -> y_pred
        # Hidden layer
        print("[x] shape: ", x.shape)
        h1 = F.conv2d(input=x, weight=w1, bias=None, stride=1, padding=0, dilation=1)
        print("[h1] shape: ", h1.shape)
        b1_resh = b1.reshape([1, b1.shape[0], 1, 1]) # Add unary dims, so that afterwards it can get broadcasted
        h1_bias = h1 + b1_resh #Unary dims (N,-,W,H) get broadcasted for the addition.
        h1_relu = h1_bias.clamp(min=0)
        print("[h1_relu] shape: ", h1_relu.shape)
        h1_pool = F.max_pool2d(h1_relu, kernel_size=[2,2], stride=[2,2], padding=0, ceil_mode=True)
        print("[h1_pool] shape: ", h1_pool.shape)
        # Fully-connected classifier (aka Densely connected classifier)
        h1_relu_flat = h1_pool.reshape( [h1_pool.shape[0], -1] )
        print("[h1_relu_flat] shape: ", h1_relu_flat.shape)
        h2 = h1_relu_flat.mm(w2)
        logits = h2 + b2.reshape([1, b2.shape[0]]) # Add bias to activation, after reshape for broadcasting.
        ############################################################################################
        
        # Softmax activation function
        exp_logits = torch.exp(logits)
        y_pred = exp_logits / torch.sum(exp_logits, dim=1, keepdim=True) 
        # sum with Keepdim=True returns [N,1] array. It would be [N] if keepdim=False.
        # Torch broadcasts [N,1] to [N,D_out] via repetition, to divide elementwise exp_h2 (which is [N,D_out]).
        
        return y_pred
    
    def backward_pass(self, loss, y_real):
        # Same as MLP in autograd.
        loss.backward()
        grads = [param.grad for param in self.params]
        return grads
        
        
########################## IMPORTANT NOTE (nothing to do) ###############################################
# The MLP receives each sample as a vector. Thus we had vectorized/flattened the 2D images prior to Task 1.
# The CNN processes 2D inputs directly. Vectorizing them is not needed.
# BUT PyTorch receives input to conv/pool of Torch needs to be of shape [N, Channels, H, W], with...
# ... second dimension being the number of input channels (number of input feature maps if hidden layer).
# For grayscale images, there is only 1 image channel. For RGB images, this would be 3. We add this dimension:
IMAGE_CHANNELS = 1
train_imgs_cnn = train_imgs.reshape([train_imgs.shape[0], IMAGE_CHANNELS, train_imgs.shape[1], train_imgs.shape[2]])
test_imgs_cnn = test_imgs.reshape([test_imgs.shape[0], IMAGE_CHANNELS, test_imgs.shape[1], test_imgs.shape[2]])
#######################################################################################################

# Create the network
rng = np.random.RandomState(seed=SEED) # Random number generator
net = CNN_Torch_Autograd(rng=rng)
# Start training
gradient_descent(net,
                 cross_entropy_torch, # Same as MLP.
                 grad_descent_update_autograd, # Same as MLP.
                 rng,
                 train_imgs_cnn,
                 train_lbls_onehot,
                 test_imgs_cnn,
                 test_lbls_onehot,
                 batch_size=40, 
                 learning_rate=1e-2,
                 total_iters=400,
                 iters_per_test=10)

If completed successfully, this basic CNN should be trainable in less than a minute on a CPU. Its testing accuracy should finish at 87.61\% for the stochastic version of gradient descent, similar to that of the MLP.


**Q: How do we compute the number of input neurons to the last hidden layer?**  
D_in_fc has been given equal to 1440. Why? How is this number computed? It is architecture dependent. To answer, consider how the shape of a tensor changes as it propagates through the CNN. How do the convs and pool operators change the shape of a tensor, with respect to their kernel size, stride, use of padding etc? This is very important to understand, to be able to implement larger and more complex networks.


### Bonus (optional):
    
Experiment yourselves with adding more conv/pool modules, to build a deeper net. Deeper networks (ala LeNet) can exceed 99\% test accuracy. Can you approach this by adding more layers?  
You will find that deeper nets not only quickly become expensive to train, but are also not easy to train. They require careful hyperparameter configuration (learning rate, batch size, initialization, total training iterations, etc). How hyper-parameters change behaviour during training is not well understood theoretically, and currently depends on practical experience. Get some if you can.


## This notebook:
Copyright 2020, Imperial College London  
Author: Konstantinos Kamnitsas (konstantinos.kamnitsas12@imperial.ac.uk)